In [61]:
import pyodbc as odbc
import os
import torch
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN
import face_recognition
import cv2
import json
mtcnn = MTCNN(keep_all=True)

In [62]:
SERVER_NAME = 'localhost\SQLEXPRESS'
DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
DATABASE_NAME = 'Face_Recognition'
cnxn = odbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; \
                    SERVER='+SERVER_NAME+'; \
                    DATABASE = '+DATABASE_NAME+'; \
                    Trusted_connection=yes;')
cur = cnxn.cursor()

In [63]:
cur.execute("use Face_Detection")

In [64]:
# CREATE TABLE All_Images
table_name = 'All_Images'
create_table_query = f"""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}')
BEGIN
    CREATE TABLE {table_name} (
        Image_ID INT PRIMARY KEY,
        Image_Path NVARCHAR(255),
        No_of_Faces INT
    );
END
"""
# Execute the query
cur.execute(create_table_query)
cur.commit()

In [65]:
# INSERT INTO All_Images TABLE
cur.execute("select max(Image_id) from all_images")
pid = cur.fetchone()
# print(pid)
if pid[0] is None:
    Id = 101
else:
    Id = pid[0] + 1
no_of_faces = 0
Image_folder = input("Enter All_Images Folder path:")
for image in os.listdir(Image_folder):
    image_path = os.path.join(Image_folder,image)

    read_image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(read_image,cv2.COLOR_BGR2RGB)
    faces,_ = mtcnn.detect(image_rgb)
    if faces is None:
        no_of_faces=str(0)
    else:
        no_of_faces = str(len(faces))
    cur.execute("select count(image_path) from all_images where image_path = ?",(image_path))
    cc = cur.fetchone()[0]
    if cc == 0:
        cur.execute("Insert into all_images(image_id,image_path,no_of_faces) values(?,?,?)",(Id,image_path,no_of_faces))
        cur.commit()
        Id+=1
    # cur.execute('MERGE INTO All_Images AS target USING (SELECT ? AS image_id, ? AS image_path , ? AS No_of_faces) AS source ON (target.image_ID = source.image_ID) WHEN NOT MATCHED THEN INSERT (image_id, image_path,no_of_faces) VALUES (source.image_id, source.image_path,source.no_of_faces);',(Id,image_path,no_of_faces))

    # cur.commit()
    # Id+=1


In [66]:
# Target_Images TABLE CREATION
table_name = 'Target_Images'
create_table_query = f"""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}')
BEGIN
    CREATE TABLE {table_name} (
        Person_ID INT PRIMARY KEY,
        Person_Image_path NVARCHAR(255),
        Name NVARCHAR(30)
    );
END
"""
# Execute the query
cur.execute(create_table_query)
cur.commit()

In [67]:
# INSERT INTO target_Image TABLE
cur.execute("select max(person_id) from Target_images")
pid = cur.fetchone()
# print(pid)
if pid[0] is None:
    Id = 1
else:
    Id = pid[0] + 1
target_image_folder = input("Enter Target_Images Folder Path:")
# target_image_folder = r"C:\Users\nihar\Desktop\4th Sem\Face\Images\Players"
for target in os.listdir(target_image_folder):
    target_image_path = os.path.join(target_image_folder,target)
    name = target.split('.')
    Name = name[0]
    cur.execute("select count(person_image_path) from target_images where person_image_path = ?",(target_image_path))
    cc = cur.fetchone()[0]
    if cc == 0:
        cur.execute("Insert into target_images(person_id,person_image_path,Name) values(?,?,?)",(Id,target_image_path,Name))
        cur.commit()
        Id+=1
    # cur.execute('MERGE INTO Target_Images AS target USING (SELECT ? AS person_id, ? AS person_image_path , ? AS Name) AS source ON (target.person_image_path = source.person_image_path) WHEN NOT MATCHED THEN INSERT (person_id, person_image_path,Name) VALUES (source.person_id, source.person_image_path,source.Name);',(Id,target_image_path,Name))

In [68]:
# MATCHED IMAGES TABLE CREATION
table_name = 'Matched_images'
create_table_query = f"""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}')
BEGIN
    CREATE TABLE {table_name} (
        Image_path nvarchar(255),
        Person_id int,
        Image_id int,
        Name nvarchar(30)
    );
END
"""
# Execute the query
cur.execute(create_table_query)
cur.commit()


In [69]:
# All_Images_ENCODINGS TABLE CREATION
table_name = 'All_Images_encodings'
create_table_query = f"""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}')
BEGIN
    CREATE TABLE {table_name} (
        Image_id int,
        encodings nvarchar(4000)
    );
END
"""
# Execute the query
cur.execute(create_table_query)
cur.commit()


In [70]:
def detect_face(image_rgb):
    boxes,_ = mtcnn.detect(image_rgb)
    face_locations=[]
    if (boxes is not None):
        boxes = list(boxes)
        for box in boxes:
            top,right,bottom,left = int(box[1]),int(box[2]),int(box[3]),int(box[0])
            face_locations.append((top,right,bottom,left))
        return face_locations
    else:
        return []

In [71]:
def encode_faces(image,face_loactions):
    face_encodings = face_recognition.face_encodings(image,face_loactions)
    return face_encodings

In [72]:
# INSERT INTO All_images_ENCODINGS TABLE
cur.execute("SELECT Image_ID,Image_path,No_of_Faces FROM All_Images")
All_images = cur.fetchall()

for image in All_images:
    read_image = cv2.imread(image[1])
    image_id = image[0]
    image_rgb = cv2.cvtColor(read_image,cv2.COLOR_BGR2RGB)
    detect_faces = detect_face(image_rgb)
    image_encode = encode_faces(image_rgb,detect_faces)
    for encoding in image_encode:
        encoding_str = json.dumps(encoding.tolist())
        cur.execute("select count(Image_ID) from All_images_encodings where Image_ID = ?",(image_id))
        c = cur.fetchone()[0]
        faces = image[2]
        if c>0 and c==faces:
            # print(faces)
            pass
            # while (c>0):
                # cur.execute("insert into All_Images_encodings(Image_id,encodings) values (?,?)",(image_id,encoding_str))
                # c = c - 1
        else:
            cur.execute("insert into All_Images_encodings(Image_id,encodings) values (?,?)",(image_id,encoding_str))
        # cur.execute("SELECT count(*) FROM All_Images_encodings WHERE encodings Like ? AND Image_ID = ?", (encoding_str,image_id))
        # row_count = cur.fetchone()[0]

        # if row_count == 0:
        # if cur.fetchone() is None:
cur.commit()


In [73]:
# Target_Images_ENCODINGS TABLE CREATION
table_name = 'Target_Images_encodings'
create_table_query = f"""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}')
BEGIN
    CREATE TABLE {table_name} (
        Person_id int,
        encodings varchar(max)
    );
END
"""
# Execute the query
cur.execute(create_table_query)
cur.commit()


In [74]:
# INSERT INTO TARGET_IMAGES_ENCODINGS TABLE
cur.execute("SELECT Person_ID,Person_Image_path FROM Target_Images")
person_images = cur.fetchall()

for image in person_images:
    person_image = cv2.imread(image[1])
    person_id = image[0]
    person_image_rgb = cv2.cvtColor(person_image,cv2.COLOR_BGR2RGB)
    detect_person = detect_face(person_image_rgb)
    person_image_encode = encode_faces(person_image_rgb,detect_person)
    for encoding in person_image_encode:
        encoding_str = json.dumps(encoding.tolist())
        cur.execute('MERGE INTO Target_Images_encodings AS target USING (SELECT ? AS Person_id, ? AS encodings) AS source ON (target.Person_id = source.Person_id) WHEN NOT MATCHED THEN INSERT (Person_id, encodings) VALUES (source.Person_id, source.encodings);',(person_id,encoding_str))
cur.commit()
